# Train MRCNN Model from pipeline
 


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
##-------------------------------------------------------------------------------------------
##
## Combined MRCNN-FCN Pipeline (import model_mrcnn)
## Train MRCNN heads only
##
##  
##-------------------------------------------------------------------------------------------

import os, sys, math, io, time
import gc
import numpy as np
import argparse
import platform
import tensorflow as tf
import keras
import keras.backend as KB
import platform

sys.path.append('../')

import mrcnn.model_mrcnn  as mrcnn_modellib
# import mrcnn.model_fcn    as fcn_modellib
import mrcnn.visualize    as visualize
import mrcnn.new_shapes   as shapes
from datetime import datetime   

from mrcnn.config       import Config
from mrcnn.dataset      import Dataset 
from mrcnn.utils        import log, stack_tensors, stack_tensors_3d, write_stdout
from mrcnn.datagen      import data_generator, load_image_gt
from mrcnn.callbacks    import get_layer_output_1,get_layer_output_2
# from mrcnn.visualize    import plot_gaussian
# from mrcnn.prep_notebook import prep_oldshapes_train, load_model

import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)
syst = platform.system()


start_time = datetime.now().strftime("%m-%d-%Y @ %H:%M:%S")
print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

##-----------------------------------------------

In [ ]:
##------------------------------------------------------------------------------------
## Parse command line arguments
##  
## Example:
##           train-shapes_gpu --epochs 12 --steps-in-epoch 7 --last_epoch 1234 --logs_dir mrcnn_logs
##------------------------------------------------------------------------------------
parser = argparse.ArgumentParser(description='Train Mask R-CNN on MS COCO.')
# parser.add_argument("command",
                    # metavar="<command>",
                    # help="'train' or 'evaluate' on MS COCO")
# parser.add_argument('--dataset', required=True,
                    # metavar="/path/to/coco/",
                    # help='Directory of the MS-COCO dataset')
# parser.add_argument('--limit', required=False,
                    # default=500,
                    # metavar="<image count>",
                    # help='Images to use for evaluation (defaults=500)')
                    
parser.add_argument('--model', required=False,
                    default='last',
                    metavar="/path/to/weights.h5",
                    help="MRCNN model weights file: 'coco' , 'init' , or Path to weights .h5 file ")

parser.add_argument('--fcn_model', required=False,
                    default='last',
                    metavar="/path/to/weights.h5",
                    help="FCN model weights file: 'init' , or Path to weights .h5 file ")

parser.add_argument('--logs_dir', required=True,
                    default='mrcnn_logs',
                    metavar="/path/to/logs/",
                    help='Logs and checkpoints directory (default=logs/)')

parser.add_argument('--last_epoch', required=False,
                    default=0,
                    metavar="<last epoch ran>",
                    help='Identify last completed epcoh for tensorboard continuation')
                    
parser.add_argument('--epochs', required=False,
                    default=3,
                    metavar="<epochs to run>",
                    help='Number of epochs to run (default=3)')
                    
parser.add_argument('--steps_in_epoch', required=False,
                    default=1,
                    metavar="<steps in each epoch>",
                    help='Number of batches to run in each epochs (default=5)')

parser.add_argument('--val_steps', required=False,
                    default=1,
                    metavar="<val steps in each epoch>",
                    help='Number of validation batches to run at end of each epoch (default=1)')
                    
parser.add_argument('--batch_size', required=False,
                    default=5,
                    metavar="<batch size>",
                    help='Number of data samples in each batch (default=5)')                    

parser.add_argument('--lr', required=False,
                    default=0.001,
                    metavar="<learning rate>",
                    help='Learning Rate (default=0.001)')
# args = parser.parse_args()


In [ ]:
# args = parser.parse_args("--epochs 100 --steps_in_epoch 128  --last_epoch 1264 --batch_size 8  --lr 0.5               --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model init".split())
args = parser.parse_args("--epochs 100 --steps_in_epoch 100  --last_epoch 1264 --batch_size 25 --lr 0.8 --val_steps 5 --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5".split())

In [ ]:

##----------------------------------------------------------------------------------------------
## if debug is true set stdout destination to stringIO
##----------------------------------------------------------------------------------------------            
debug = False
if debug:
    sys.stdout = io.StringIO()

In [ ]:
print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

# print("Dataset: ", args.dataset)
# print("Logs:    ", args.logs)
# print("Limit:   ", args.limit)

print("    MRCNN Model        : ", args.model)
print("    FCN Model          : ", args.fcn_model)
print("    Log Dir            : ", args.logs_dir)
print("    Last Epoch         : ", args.last_epoch)
print("    Epochs to run      : ", args.epochs)
print("    Steps in each epoch: ", args.steps_in_epoch)
print("    Validation steps   : ", args.val_steps)
print("    Batch Size         : ", args.batch_size)
print("    OS Platform        : ", syst)


In [ ]:
##------------------------------------------------------------------------------------
## setup project directories
#---------------------------------------------------------------------------------
# # Root directory of the project 
# MODEL_DIR    :    Directory to save logs and trained model
# COCO_MODEL_PATH  : Path to COCO trained weights
#---------------------------------------------------------------------------------
import platform
syst = platform.system()
if syst == 'Windows':
    # WINDOWS MACHINE ------------------------------------------------------------------
    ROOT_DIR          = "E:\\"
    MODEL_PATH        = os.path.join(ROOT_DIR    , "models")
    DATASET_PATH      = os.path.join(ROOT_DIR    , 'MLDatasets')
    MODEL_DIR         = os.path.join(MODEL_PATH  , args.logs_dir)
    COCO_MODEL_PATH   = os.path.join(MODEL_PATH  , "mask_rcnn_coco.h5")
    DEFAULT_LOGS_DIR  = os.path.join(MODEL_PATH  , args.logs_dir) 
    COCO_DATASET_PATH = os.path.join(DATASET_PATH, "coco2014")
    RESNET_MODEL_PATH = os.path.join(MODEL_PATH  , "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")
    VGG16_MODEL_PATH  = os.path.join(MODEL_PATH  , "fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5")
    
elif syst == 'Linux':
    # LINUX MACHINE ------------------------------------------------------------------
    ROOT_DIR          = os.getcwd()
    MODEL_PATH        = os.path.expanduser('~/models')
    DATASET_PATH      = os.path.expanduser('~/MLDatasets')
    MODEL_DIR         = os.path.join(MODEL_PATH  , args.logs_dir)
    COCO_MODEL_PATH   = os.path.join(MODEL_PATH  , "mask_rcnn_coco.h5")
    COCO_DATASET_PATH = os.path.join(DATASET_PATH, "coco2014")
    DEFAULT_LOGS_DIR  = os.path.join(MODEL_PATH  , args.logs_dir)
    RESNET_MODEL_PATH = os.path.join(MODEL_PATH  , "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")
    VGG16_MODEL_PATH  = os.path.join(MODEL_PATH  , "fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5")
else :
    raise Error('unreconized system  '      )



In [ ]:
##------------------------------------------------------------------------------------
## Build configuration object 
##------------------------------------------------------------------------------------
mrcnn_config                    = shapes.NewShapesConfig()
mrcnn_config.BATCH_SIZE         = int(args.batch_size)                  # Batch size is 2 (# GPUs * images/GPU).
mrcnn_config.IMAGES_PER_GPU     = int(args.batch_size)                  # Must match BATCH_SIZE
mrcnn_config.STEPS_PER_EPOCH    = int(args.steps_in_epoch)
mrcnn_config.LEARNING_RATE      = float(args.lr)
mrcnn_config.EPOCHS_TO_RUN      = int(args.epochs)
mrcnn_config.FCN_INPUT_SHAPE    = mrcnn_config.IMAGE_SHAPE[0:2]
mrcnn_config.LAST_EPOCH_RAN     = int(args.last_epoch)
mrcnn_config.WEIGHT_DECAY       = 2.0e-4
mrcnn_config.VALIDATION_STEPS   = int(args.val_steps)
mrcnn_config.REDUCE_LR_FACTOR   = 0.5
mrcnn_config.REDUCE_LR_COOLDOWN = 30
mrcnn_config.REDUCE_LR_PATIENCE = 40
mrcnn_config.EARLY_STOP_PATIENCE= 80
mrcnn_config.EARLY_STOP_MIN_DELTA = 1.0e-4
mrcnn_config.MIN_LR             = 1.0e-10
mrcnn_config.NEW_LOG_FOLDER     = True  
mrcnn_config.display() 


In [ ]:
##------------------------------------------------------------------------------------
## Build shape dataset for Training and Validation       
##------------------------------------------------------------------------------------
dataset_train = shapes.NewShapesDataset(mrcnn_config)
dataset_train.load_shapes(10000)
dataset_train.prepare()

dataset_val = shapes.NewShapesDataset(mrcnn_config)
dataset_val.load_shapes(2500)
dataset_val.prepare()


In [ ]:
##------------------------------------------------------------------------------------
## Build Mask RCNN Model in TRAINING mode
##------------------------------------------------------------------------------------
try :
    del mrcnn_model
    gc.collect()
except: 
    pass
KB.clear_session()
mrcnn_model = mrcnn_modellib.MaskRCNN(mode="training", config=mrcnn_config, model_dir=MODEL_DIR, FCN_layers = False)

In [ ]:
##------------------------------------------------------------------------------------
## Load Mask RCNN Model Weight file
##------------------------------------------------------------------------------------
# exclude_list = ["mrcnn_class_logits"]
#load_model(model, init_with = args.model)   
exclude_list = []
mrcnn_model.load_model_weights(init_with = args.model, exclude = exclude_list)   

print('==========================================')
print(" MRCNN MODEL Load weight file COMPLETE    ")
print('==========================================')

mrcnn_config.display()  
mrcnn_model.layer_info()

print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)
print('config.BATCH_SIZE      : ', config.BATCH_SIZE)
print('model.config.BATCH_SIZE: ', model.config.BATCH_SIZE)
# exit(8)

## Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_2500.h5")
model.keras_model.save_weights(model_path)